In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import defaultdict

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train=train.drop('keyword',1)
train=train.drop('location',1)

test=test.drop('keyword',1)
test=test.drop('location',1)

In [5]:
test.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
y_train=train.iloc[:,-1]
x_train=train.iloc[:,:-1]

In [7]:
ps=PorterStemmer()

In [8]:
def create_corpus(data):
    corpus=[]
    for i in range(data.shape[0]):
        tweet=data.iloc[i,-1]
        tweet=re.sub('[^a-zA-z]'," ",tweet)
        tweet=tweet.lower()
        tweet=tweet.split()
        tweet=[ps.stem(word) for word in tweet if word not in stopwords.words('english')]
        tweet=" ".join(tweet)
        corpus.append(tweet)
    return corpus

In [9]:
x_corpus_train=create_corpus(x_train)
x_corpus_test=create_corpus(test)

In [10]:
dic=defaultdict(int)
for text in x_corpus_train:
    text=text.split()
    for word in text:
        dic[word]=dic[word]+1

In [11]:
# sorted(dic.items(), key=lambda x:x[1],reverse=True)
len(dic)

19043

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)

In [13]:
x_train_vector=cv.fit_transform(x_corpus_train).toarray()
x_test_vector=cv.transform(x_corpus_test).toarray()

In [14]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train_vector,y_train)
print(model.score(x_train_vector,y_train))
y_pred=model.predict(x_test_vector)
y_pred

0.8591882306580848


array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [17]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=4,n_estimators=500,warm_start=True,max_depth=6,min_samples_leaf=2,max_features='auto',min_samples_split=3)
rfc.fit(x_train_vector,y_train)
print(rfc.score(x_train_vector,y_train))
y_pred=rfc.predict(x_test_vector)

0.6385130697491134


In [21]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(x_train_vector,y_train,early_stopping_rounds=5, 
             eval_set=[(x_train_vector,y_train)], 
             verbose=False)
print(xgb.score(x_train_vector,y_train))
y_pred=xgb.predict(x_test_vector)

0.8398791540785498


In [22]:
from sklearn.linear_model import LogisticRegression
reg=LogisticRegression()
reg.fit(x_train_vector,y_train)
print(reg.score(x_train_vector,y_train))
y_pred=reg.predict(x_test_vector)
y_pred

0.919873899908052


array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [26]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 2)
classifier.fit(x_train_vector,y_train)
print(classifier.score(x_train_vector,y_train))
y_pred=classifier.predict(x_test_vector)

0.7992906869827926


In [27]:
submission=pd.read_csv('sample_submission.csv')
submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [28]:
submission['target']=y_pred

In [29]:
submission.to_csv('submission.csv',index=False)